In [1]:
# -*- coding: utf-8 -*-

In [2]:
from cnn_model import *
from Generate_Captcha import *
import os
from PIL import Image
from datetime import datetime

In [3]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [4]:
class Run:

    def __init__(self):
        read = ReadData()
        self.test_x, self.test_y, self.test_num = read.load_data(folder=Config.test_folder)
        self.train_x, self.train_y, self.train_num = read.load_data(folder=Config.train_folder)
        self.val_x, self.val_y, self.val_num = read.load_data(folder=Config.validation_folder)

        print 'Images for train ：%d, for validation : %d, for test : %d' \
              % (self.train_num, self.val_num, self.test_num)

        self.run_model()

    @staticmethod
    def next_batch(x, y, length):
        if length % Config.batch_size == 0:
            times = int(length / Config.batch_size)
        else:
            times = int(length / Config.batch_size) + 1

        start_id = 0
        for _ in range(times):
            end_id = min(start_id + Config.batch_size, length)
            batch_data = x[start_id:end_id]
            batch_label = y[start_id:end_id]
            start_id = end_id
            yield batch_data, batch_label

    @staticmethod
    def feed_data(x, y, keep_prob, is_training=True):
        feed_dict = {model.input_x: x,
                     model.input_y: y,
                     model.keep_prob: keep_prob,
                     model.training: is_training}
        return feed_dict

    def evaluate(self, sess, val_x, val_y, val_size):
        total_loss = 0.
        total_acc = 0.

        for x_, y_ in self.next_batch(val_x, val_y, val_size):
            length = len(y_)
            feed_dict = self.feed_data(x_, y_, 1.0, False)
            val_acc, val_loss = sess.run([model.accuracy, model.loss], feed_dict=feed_dict)
            total_acc += val_acc * length
            total_loss += val_loss * length
        return total_acc / val_size, total_loss / val_size

    def run_model(self):

        saver = tf.train.Saver(max_to_keep=1)
        if not os.path.exists(Config.saver_folder):
            os.mkdir(Config.saver_folder)
        save_path = os.path.join(Config.saver_folder, 'best_validation')

        total_batch = 0
        best_acc = 0
        last_improved_step = 0
        require_steps = 1000
        flag = False
        start_time = datetime.now()

        sess = tf.Session()
        sess.run(tf.global_variables_initializer())

        for epoch in range(Config.Epoch):
            print 'Epoch : %d' % (epoch + 1)
            for x, y in self.next_batch(self.train_x, self.train_y, self.train_num):
                feed_dict = self.feed_data(x, y, Config.keep_prob, True)
                sess.run(model.train_step, feed_dict=feed_dict)

                if total_batch % Config.print_per_batch == 0:
                    # Output the accuracy and loss values on the validation and training sets
                    feed_dict[model.keep_prob] = 1.0
                    feed_dict[model.training] = False
                    train_accuracy, train_loss = sess.run([model.accuracy, model.loss],
                                                          feed_dict=feed_dict)
                    val_acc, val_loss = self.evaluate(sess, self.val_x, self.val_y, self.val_num)

                    if val_acc > best_acc:
                        # record the best result
                        best_acc = val_acc
                        last_improved_step = total_batch
                        # store the model
                        saver.save(sess=sess, save_path=save_path)
                        improved = '*'
                    else:
                        improved = ''

                    msg = 'Step {:5}, train_acc:{:8.2%}, train_loss:{:6.2f},' \
                          ' val_acc:{:8.2%}, val_loss:{:6.2f}, improved:{:3}'
                    print msg.format(total_batch, train_accuracy, train_loss, val_acc, val_loss, improved)

                if total_batch % Config.save_per_batch == 0:
                    # write in tensorboard
                    feed_dict[model.keep_prob] = 1.0
                    feed_dict[model.training] = False
                    s = sess.run(model.merged_summary, feed_dict=feed_dict)
                    model.writer.add_summary(s, total_batch)

                if total_batch - last_improved_step > require_steps:
                    flag = True
                    break

                total_batch += 1
            if flag:
                print 'No improvement for over %d steps, auto-stopping....' % require_steps
                break
        end_time = datetime.now()
        time_diff = (end_time - start_time).seconds
        print 'Time Usage : {:.2f} hours'.format(time_diff / 3600.0)
        # print the accuracy on the test set
        test_acc, test_loss = self.evaluate(sess, self.test_x, self.test_y, self.test_num)

        print "Test accuracy:{:8.2%}, loss:{:6.2f}".format(test_acc, test_loss)
        sess.close()

In [5]:
if __name__ == "__main__":
    model = CNN()
    Run()

Instructions for updating:
Use keras.layers.batch_normalization instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Images for train ：10000, for validation : 500, for test : 500
Epoch : 1
Step     0, train_acc:  12.11%, train_loss:  1.97, val_acc:  10.20%, val_loss:  2.01, improved:*  
Step    20, train_acc:   7.81%, train_loss:  0.48, val_acc:   9.70%, val_loss:  0.48, improved:   
Step    40, train_acc:  10.94%, train_loss:  0.38, val_acc:   9.20%, val_loss:  0.39, improved:   
Step    60, train_acc:  12.11%, train_loss:  0.35, val_acc:   8.70%, val_loss:  0.36, improved:   
Step    80, train_acc:  12.11%, train_loss:  0.34, val_acc:   9.10%, val_loss:  0.34, improved:   
Step   100, train_acc:  11.72%, train_loss:  0.33, val_acc:   9.45%, val_loss:  0.33, improved:   
Step   120, train_acc:   9.38%, train_loss:  0.33, val_acc:   8.55%, val_l

Step  1460, train_acc:  75.00%, train_loss:  0.18, val_acc:  69.40%, val_loss:  0.19, improved:*  
Step  1480, train_acc:  76.56%, train_loss:  0.17, val_acc:  69.40%, val_loss:  0.19, improved:*  
Step  1500, train_acc:  73.83%, train_loss:  0.18, val_acc:  69.85%, val_loss:  0.18, improved:*  
Step  1520, train_acc:  75.39%, train_loss:  0.18, val_acc:  70.30%, val_loss:  0.19, improved:*  
Step  1540, train_acc:  74.22%, train_loss:  0.18, val_acc:  69.40%, val_loss:  0.18, improved:   
Step  1560, train_acc:  66.41%, train_loss:  0.18, val_acc:  71.75%, val_loss:  0.18, improved:*  
Epoch : 11
Step  1580, train_acc:  73.05%, train_loss:  0.18, val_acc:  71.10%, val_loss:  0.18, improved:   
Step  1600, train_acc:  69.14%, train_loss:  0.18, val_acc:  70.45%, val_loss:  0.18, improved:   
Step  1620, train_acc:  77.73%, train_loss:  0.17, val_acc:  71.50%, val_loss:  0.18, improved:   
Step  1640, train_acc:  76.95%, train_loss:  0.17, val_acc:  70.70%, val_loss:  0.18, improved:   

Step  3100, train_acc:  92.19%, train_loss:  0.07, val_acc:  88.05%, val_loss:  0.09, improved:   
Step  3120, train_acc:  95.31%, train_loss:  0.06, val_acc:  89.15%, val_loss:  0.09, improved:*  
Epoch : 21
Step  3140, train_acc:  94.92%, train_loss:  0.06, val_acc:  89.05%, val_loss:  0.09, improved:   
Step  3160, train_acc:  96.09%, train_loss:  0.06, val_acc:  89.55%, val_loss:  0.09, improved:*  
Step  3180, train_acc:  95.31%, train_loss:  0.06, val_acc:  88.80%, val_loss:  0.09, improved:   
Step  3200, train_acc:  95.31%, train_loss:  0.06, val_acc:  89.60%, val_loss:  0.09, improved:*  
Step  3220, train_acc:  96.48%, train_loss:  0.05, val_acc:  89.55%, val_loss:  0.09, improved:   
Step  3240, train_acc:  95.70%, train_loss:  0.05, val_acc:  89.35%, val_loss:  0.08, improved:   
Step  3260, train_acc:  93.36%, train_loss:  0.06, val_acc:  90.20%, val_loss:  0.09, improved:*  
Step  3280, train_acc:  95.31%, train_loss:  0.05, val_acc:  90.25%, val_loss:  0.08, improved:*  

Step  4740, train_acc:  98.83%, train_loss:  0.02, val_acc:  94.25%, val_loss:  0.06, improved:   
Step  4760, train_acc:  97.27%, train_loss:  0.03, val_acc:  93.80%, val_loss:  0.06, improved:   
Step  4780, train_acc:  98.44%, train_loss:  0.02, val_acc:  93.50%, val_loss:  0.06, improved:   
Step  4800, train_acc:  98.44%, train_loss:  0.02, val_acc:  94.35%, val_loss:  0.06, improved:   
Step  4820, train_acc:  99.22%, train_loss:  0.02, val_acc:  95.05%, val_loss:  0.06, improved:*  
Step  4840, train_acc:  98.44%, train_loss:  0.03, val_acc:  94.20%, val_loss:  0.06, improved:   
Step  4860, train_acc:  98.83%, train_loss:  0.02, val_acc:  93.70%, val_loss:  0.06, improved:   
Epoch : 32
Step  4880, train_acc:  99.61%, train_loss:  0.02, val_acc:  94.25%, val_loss:  0.05, improved:   
Step  4900, train_acc:  98.05%, train_loss:  0.02, val_acc:  94.25%, val_loss:  0.06, improved:   
Step  4920, train_acc:  99.61%, train_loss:  0.02, val_acc:  94.20%, val_loss:  0.06, improved:   

Step  6380, train_acc:  99.22%, train_loss:  0.01, val_acc:  95.15%, val_loss:  0.06, improved:   
Step  6400, train_acc:  99.22%, train_loss:  0.01, val_acc:  95.35%, val_loss:  0.05, improved:   
Step  6420, train_acc: 100.00%, train_loss:  0.01, val_acc:  94.45%, val_loss:  0.06, improved:   
Epoch : 42
Step  6440, train_acc:  99.61%, train_loss:  0.01, val_acc:  94.95%, val_loss:  0.06, improved:   
Step  6460, train_acc: 100.00%, train_loss:  0.01, val_acc:  95.05%, val_loss:  0.05, improved:   
Step  6480, train_acc: 100.00%, train_loss:  0.01, val_acc:  95.15%, val_loss:  0.05, improved:   
Step  6500, train_acc:  99.61%, train_loss:  0.01, val_acc:  94.50%, val_loss:  0.06, improved:   
Step  6520, train_acc:  99.22%, train_loss:  0.01, val_acc:  94.85%, val_loss:  0.06, improved:   
Step  6540, train_acc:  99.61%, train_loss:  0.01, val_acc:  95.10%, val_loss:  0.06, improved:   
Step  6560, train_acc: 100.00%, train_loss:  0.01, val_acc:  94.95%, val_loss:  0.05, improved:   

Epoch : 52
Step  8020, train_acc: 100.00%, train_loss:  0.00, val_acc:  95.30%, val_loss:  0.06, improved:   
Step  8040, train_acc: 100.00%, train_loss:  0.01, val_acc:  95.60%, val_loss:  0.06, improved:   
Step  8060, train_acc: 100.00%, train_loss:  0.01, val_acc:  95.30%, val_loss:  0.06, improved:   
Step  8080, train_acc: 100.00%, train_loss:  0.01, val_acc:  95.40%, val_loss:  0.06, improved:   
Step  8100, train_acc: 100.00%, train_loss:  0.01, val_acc:  95.55%, val_loss:  0.06, improved:   
Step  8120, train_acc: 100.00%, train_loss:  0.01, val_acc:  95.45%, val_loss:  0.05, improved:   
Step  8140, train_acc: 100.00%, train_loss:  0.00, val_acc:  95.55%, val_loss:  0.06, improved:   
Step  8160, train_acc: 100.00%, train_loss:  0.01, val_acc:  95.50%, val_loss:  0.06, improved:   
Epoch : 53
Step  8180, train_acc: 100.00%, train_loss:  0.00, val_acc:  95.65%, val_loss:  0.05, improved:   
Step  8200, train_acc: 100.00%, train_loss:  0.00, val_acc:  95.60%, val_loss:  0.05, i